* * *
<pre> NYU Paris            <i> Artificial intelligence - Summer 2023 </i></pre>
* * *


<h1 align="center"> Lab 8: Unsupervised learning (II): similarity </h1>

<pre align="left"> October 26th 2023               <i> Author: Hicham Janati </i></pre>
* * *


##### Goals:
- Understand the difference between distance and similarity
- Apply this concept to face recognition 

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
root = "/content/drive/MyDrive/Data/facerecognition"
sys.path.append(root)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Part 1: Introduction 
To explain the notion of similarity, consider the following example. We want to aggregate documents based on their topics. For the sake of simplicity, we take documents made of only a few sentences. The following cell reads the documents.

In [ ]:
with open(f"{root}/text/nfts.txt", "r") as docsfile:
    documents = docsfile.read().split('\n---\n')
print(f"Number of documents: {len(documents)}")
print(f"First document:\n\n {documents[0][:500]}")

### Question 1
The top 2 common themes of these articles are NFTs and football. We want to classify them based on the extent to which these themes are present. To do so, write a function that takes a document in argument (a string) and returns a vectors containing the number of times the words "nft", "nfts", "football", "footballer" are present, in that order. 

For example, `text_to_vec("a footballer bought two NFTs: 1 digital art NFT and 1 music NFT")` should return: `[2, 1, 0, 1]`.

*Hint: Check the documentation of these string and list methods: (`string.lower` `string.split` `list.count`)*

In [ ]:
def text_to_vec(text, dictionary=["nft", "nfts", "football", "footballer", "footballers"]):

    return counts

### Question 2
To make things simple, modify the function `text_to_vec` so that it aggregates the counts for ["nft, "nfts"] and ["football", "footballer", "footballers"]. The same example above should then return: `[3, 2]`. Then transform the list of documents to a list of vectors. 

### Question 3
Print the vector of each documents. Intuitively, based only on these counts what documents do you consider similar / dissimilar ? Use the function below to visualize the vectors. Is this visualization in agreement with your intuition ? 

In [ ]:
def plot_vectors(vectors):
    colors = ["indianred", "purple", "orange", "forestgreen", "navy"]
    plt.figure(figsize=(6, 6))
    for ii, vec in enumerate(vectors):
        plt.arrow(0,0,vec[0] * 0.95, vec[1] * 0.97, 
        shape='full', color=colors[ii], length_includes_head=True, 
        zorder=0, head_length=0.2, head_width=0.2, alpha=0.6, lw=2)
        plt.scatter(vec[0], vec[1], marker=rf"${ii}$", color=colors[ii], s=200)
    plt.grid()
    plt.xlim([-1, None])
    plt.show()

### Question 4 
Normalize the vectors by their norm so that their Euclidean norm $\|.\|_2$ is equal to 1. Visualize the new vectors. What to you observe ?

### Question 5
Compute the Euclidean distance between two unit-norm vectors $x$ and $y$. Can you deduce from it a notion of similarity between two vectors $x$ and $y$ ?

# Part 2: Similarity score for face recognition

The following cell reads the facial images dataset of celebrities and images of non-celebrities, here referred to as "impostors". 

In [ ]:
import cv2
import torch
import glob

labels = []
imgs = []
celebs = ["thorne", "mac", "kendrick", "jolie", "hathaway", "arnold", "benedict", "impostors"]
genders_list = [1, 0, 1, 1, 1, 0, 0, None]
genders = []


for label, name in enumerate(celebs):
    imgpath = f"{root}/imgs/{name}/"
    gender = genders_list[label]
    for ii, filename in enumerate(glob.glob(imgpath + "*.jpg")):
        img = cv2.imread(filename)
        if img is None:
            print(filename)
            break
        img = cv2.resize(img, (112, 112))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = np.transpose(img, (2, 0, 1))
        img = torch.from_numpy(img).float()
        img.div_(255).sub_(0.5).div_(0.5)
        imgs.append(img)
        labels.append(label)
        if gender is not None:
            genders.append(gender)

with open(f"{root}/imgs/gender.txt", "r") as genderfile:
    gender_imp = genderfile.read().split(',\n')
gender_imp = np.array(gender_imp).astype(int)

genders.extend(gender_imp)
imgs = torch.stack(imgs)
labels = np.array(labels)

labels[labels < len(celebs) - 1] = 0
labels[labels == len(celebs) - 1] = 1
genders = np.array(genders)
labels = torch.tensor(labels)

print(imgs.shape, labels.shape, genders.shape)
print("pct of imposters = ", 100 * labels.float().mean().item())
print("pct of males = ", 100 * (1- genders.mean()))

An embedding is a neural network transforming a sample (image, text, data ..) to multi-dimensional space -- generally with a lower dimension than the input space. Such that this new representation is more "meaningful". For example, a good text embedding would provide a representation where words having a close meaning would be geometrically closer. Here we use an embedding for facial recognition which is already pre-trained i.e we only read the weights which are already optimized. 

In general, the goal is to learn a neural net such that by computing the similarity score, it is very easy to detect impostors from genuine samples.

In [ ]:
import torch
from backbones import get_model
modelpath = f"{root}/models/backbone-r100.pth"
network = "r100"
output_dir = "outputs/"
batch_size = 128
device = "cpu"
# load backbone
model = get_model(network, fp16=False)
model.load_state_dict(torch.load(modelpath, map_location='cpu'))
model.eval()

### Question 6
Use the loaded pre-trained model to compute the embedding of all the images. The resulting object should be one matrix containing the embeddings of all images. Use GPUs for speed-up.

### Question 7
Compute the pairwise matrices of Euclidean distances and similarity scores. Check the documentation of `sklearn.metrics.pairwise` for help. Visualize the obtained matrices with `plt.imshow`. Add a `plt.colorbar()`. What do you notice ?

### Question 8
The following cell computes the correct acceptance rate as a function of the false acceptance rate while varying the threshold on the cosine similarity score for both males and females. What do you conclude ?

In [ ]:
from evaluation import Evaluation
from metrics import Metrics


eval = Evaluation(embeddings, labels, genders)
scores = eval.eval(model)
Metrics(scores).plot_roc()